In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
import urllib

# Define the path to the Delta table
delta_table_path = "dbfs:/user/hive/warehouse/authentication_credentials"

# Read the Delta table to a Spark DataFrame
aws_keys_df = spark.read.format("delta").load(delta_table_path)

In [0]:
# Get the AWS access key and secret key from the spark dataframe
ACCESS_KEY = aws_keys_df.select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df.select('Secret access key').collect()[0]['Secret access key']
# Encode the secrete key
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")

In [0]:
# Define the S3 bucket and mount point
S3_BUCKET = "user-0affd9571f39-bucket"
USER_ID = "0affd9571f39"

# Define paths to JSON files
pin_path = f"s3a://{S3_BUCKET}/topics/{USER_ID}.pin/partition=0/"
geo_path = f"s3a://{S3_BUCKET}/topics/{USER_ID}.geo/partition=0/"
user_path = f"s3a://{S3_BUCKET}/topics/{USER_ID}.user/partition=0/"


In [0]:
# Read Pinterest post data
df_pin = spark.read.json(pin_path)
df_pin.show()  # Display the Pinterest post data

# Read Geolocation data
df_geo = spark.read.json(geo_path)
df_geo.show()  # Display the Geolocation data

# Read User data
df_user = spark.read.json(user_path)
df_user.show()  # Display the User data


+--------------+--------------------+----------+--------------+--------------------+-----+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|      category|         description|downloaded|follower_count|           image_src|index|is_image_or_video|         poster_name|       save_location|            tag_list|               title|           unique_id|
+--------------+--------------------+----------+--------------+--------------------+-----+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|     christmas|My favorite 75+ N...|         1|           31k|https://i.pinimg....| 2604|            image|  Everyday Wholesome|Local save in /da...|Colorful Christma...|75+ Neutral Chris...|087b0fa9-f901-426...|
|        travel|See families trav...|         1|            9k|https://i.pinimg....|10119|            image|OUR NEXT ADVENTUR...|Local save in /

In [0]:
from pyspark.sql.functions import col, when, regexp_replace, udf
from pyspark.sql.types import IntegerType

# 1. Replace empty entries and entries with no relevant data with None
df_pin_cleaned = df_pin.replace(["", " ", "null", "N/A"], None)

# 2. Transform `follower_count` to ensure every entry is a number
# Handle cases where follower_count contains strings like "1k" or "1M"
def parse_follower_count(follower_count):
    if follower_count is None:
        return None
    if isinstance(follower_count, str):
        if "k" in follower_count.lower():
            return int(float(follower_count.lower().replace("k", "")) * 1000)
        elif "m" in follower_count.lower():
            return int(float(follower_count.lower().replace("m", "")) * 1000000)
    try:
        return int(follower_count)
    except ValueError:
        return None

parse_follower_count_udf = udf(parse_follower_count, IntegerType())

df_pin_cleaned = df_pin_cleaned.withColumn(
    "follower_count", parse_follower_count_udf(col("follower_count"))
)

# 3. Ensure numeric columns have numeric data types
numeric_columns = ["follower_count"]  # Add other numeric columns if necessary
for column in numeric_columns:
    df_pin_cleaned = df_pin_cleaned.withColumn(column, col(column).cast("int"))

# 4. Clean `save_location` to include only the save location path
# Assuming `save_location` has some unwanted prefixes or suffixes to clean
df_pin_cleaned = df_pin_cleaned.withColumn(
    "save_location", regexp_replace(col("save_location"), r"unwanted_pattern", "")
)

# 5. Rename the `index` column to `ind`
df_pin_cleaned = df_pin_cleaned.withColumnRenamed("index", "ind")

# 6. Reorder columns to the desired order
desired_column_order = [
    "ind",
    "unique_id",
    "title",
    "description",
    "follower_count",
    "poster_name",
    "tag_list",
    "is_image_or_video",
    "image_src",
    "save_location",
    "category",
]

df_pin_cleaned = df_pin_cleaned.select(*desired_column_order)

# Show the cleaned DataFrame
df_pin_cleaned.show()


+-----+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------+
|  ind|           unique_id|               title|         description|follower_count|         poster_name|            tag_list|is_image_or_video|           image_src|       save_location|      category|
+-----+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------+
| 2604|087b0fa9-f901-426...|75+ Neutral Chris...|My favorite 75+ N...|         31000|  Everyday Wholesome|Colorful Christma...|            image|https://i.pinimg....|Local save in /da...|     christmas|
|10119|40eab9ba-7812-4f2...|How to Afford Fam...|See families trav...|          9000|OUR NEXT ADVENTUR...|Family Vacation D...|            image|https://i.pinimg....|Local save in /da...| 

In [0]:
from pyspark.sql.functions import array, to_timestamp

# 1. Create a new column `coordinates` that contains an array based on the latitude and longitude columns
df_geo_cleaned = df_geo.withColumn("coordinates", array("latitude", "longitude"))

# 2. Drop the `latitude` and `longitude` columns
df_geo_cleaned = df_geo_cleaned.drop("latitude", "longitude")

# 3. Convert the `timestamp` column from a string to a timestamp data type
df_geo_cleaned = df_geo_cleaned.withColumn("timestamp", to_timestamp("timestamp"))

# 4. Reorder the DataFrame columns to have the desired column order
desired_column_order = ["ind", "country", "coordinates", "timestamp"]
df_geo_cleaned = df_geo_cleaned.select(*desired_column_order)

# Show the cleaned DataFrame
df_geo_cleaned.show()


+-----+--------------------+--------------------+-------------------+
|  ind|             country|         coordinates|          timestamp|
+-----+--------------------+--------------------+-------------------+
| 2301|Saint Vincent and...|  [13.4683, 51.7244]|2020-11-14 00:25:28|
| 2293|British Virgin Is...|[-87.7946, -159.647]|2022-03-21 10:46:53|
| 2604| Antigua and Barbuda|[-80.8933, -104.972]|2018-12-01 09:23:35|
|  603|Netherlands Antilles| [14.0083, -141.603]|2019-06-25 05:13:01|
|10119|    Christmas Island|[-74.5431, -162.795]|2020-10-22 01:59:58|
| 2060| Trinidad and Tobago|  [52.4584, 68.6527]|2020-01-25 13:54:17|
| 6261|    Marshall Islands|[-10.3101, -109.763]|2021-11-11 16:18:45|
| 3201|    Pitcairn Islands| [34.0532, -68.4946]|2019-06-25 08:31:37|
| 1422|      American Samoa|[-88.5252, -172.436]|2018-04-30 08:27:21|
| 8312|      American Samoa|[-77.9744, -106.258]|2021-04-25 15:56:29|
| 1450|      American Samoa|[-88.5252, -172.436]|2022-01-15 16:04:27|
| 8578|       Guinea

In [0]:
from pyspark.sql.functions import concat_ws, to_timestamp

# 1. Create a new column `user_name` by concatenating `first_name` and `last_name`
df_user_cleaned = df_user.withColumn("user_name", concat_ws(" ", "first_name", "last_name"))

# 2. Drop the `first_name` and `last_name` columns
df_user_cleaned = df_user_cleaned.drop("first_name", "last_name")

# 3. Convert the `date_joined` column from a string to a timestamp data type
df_user_cleaned = df_user_cleaned.withColumn("date_joined", to_timestamp("date_joined"))

# 4. Reorder the DataFrame columns to have the desired column order
desired_column_order = ["ind", "user_name", "age", "date_joined"]
df_user_cleaned = df_user_cleaned.select(*desired_column_order)

# Show the cleaned DataFrame
df_user_cleaned.show()


+-----+--------------------+---+-------------------+
|  ind|           user_name|age|        date_joined|
+-----+--------------------+---+-------------------+
| 2015|Christopher Bradshaw| 27|2016-03-08 13:38:37|
|10673| Alexander Cervantes| 59|2017-05-12 21:22:17|
| 6398| Christina Davenport| 39|2016-06-29 20:43:59|
| 3599| Alexandria Alvarado| 20|2015-10-23 04:13:23|
|10509|   Brittany Thompson| 49|2016-04-22 20:36:02|
|10119|    Chelsea Gonzalez| 43|2016-07-21 15:25:08|
| 7768|    Christine Cortez| 23|2015-12-01 18:15:02|
| 8930|     Andrew Anderson| 23|2015-11-28 11:52:37|
| 6566|     Alexander Perez| 31|2017-08-04 14:30:22|
| 3729|     Richard Edwards| 52|2016-02-07 20:00:25|
|10552|      Michael Hunter| 40|2017-05-16 07:09:21|
| 9074|     Aaron Alexander| 21|2015-10-25 07:36:08|
| 3201|     Elizabeth Terry| 28|2016-01-29 20:32:05|
|  771|     Brittany Butler| 32|2016-03-10 04:11:31|
| 1545|      Jonathan Avila| 46|2015-11-27 23:11:21|
|  603|     Brandon Jackson| 44|2016-10-29 02:

In [0]:
from pyspark.sql import Window
from pyspark.sql.functions import col, count, rank

# Join df_pin and df_geo on 'ind'
df_combined = df_pin_cleaned.join(df_geo_cleaned, on="ind", how="inner")

# Group by country and category, and count the number of posts
category_counts = df_combined.groupBy("country", "category").agg(
    count("*").alias("category_count")
)

# Define a window partitioned by country and ordered by category_count in descending order
window_spec = Window.partitionBy("country").orderBy(col("category_count").desc())

# Add a rank column to identify the top category per country
ranked_categories = category_counts.withColumn("rank", rank().over(window_spec))

# Filter to only include the most popular category (rank = 1) per country
most_popular_categories = ranked_categories.filter(col("rank") == 1).drop("rank")

# Show the results
most_popular_categories.show()


+-------------------+--------------+--------------+
|            country|      category|category_count|
+-------------------+--------------+--------------+
|        Afghanistan|     education|             1|
|        Afghanistan|           art|             1|
|            Albania|           art|             1|
|            Albania|    home-decor|             1|
|            Algeria|        quotes|             2|
|     American Samoa|        beauty|             2|
|            Andorra|       tattoos|             1|
|            Andorra|           art|             1|
|            Andorra|        beauty|             1|
|           Anguilla|        beauty|             1|
|Antigua and Barbuda|     christmas|             1|
|          Argentina|diy-and-crafts|             1|
|          Argentina|        beauty|             1|
|          Argentina|       tattoos|             1|
|            Armenia|     christmas|             1|
|            Armenia|        beauty|             1|
|           

In [0]:
from pyspark.sql.functions import year, col, count

# Extract the year from the timestamp column
df_pin_with_year = df_combined.withColumn("post_year", year("timestamp"))

# Filter posts between 2018 and 2022
df_filtered = df_pin_with_year.filter((col("post_year") >= 2018) & (col("post_year") <= 2022))

# Group by post_year and category, and count the number of posts
df_category_count = df_filtered.groupBy("post_year", "category").agg(
    count("*").alias("category_count")
)

# Order the results for better readability
df_category_count = df_category_count.orderBy("post_year", "category")

# Show the resulting DataFrame
df_category_count.show()


+---------+--------------+--------------+
|post_year|      category|category_count|
+---------+--------------+--------------+
|     2018|           art|             3|
|     2018|        beauty|             3|
|     2018|     christmas|             2|
|     2018|diy-and-crafts|             1|
|     2018|     education|             2|
|     2018|       finance|             1|
|     2018|  mens-fashion|             1|
|     2018|        quotes|             1|
|     2018|       tattoos|             1|
|     2018|        travel|             2|
|     2018|      vehicles|             1|
|     2019|           art|             2|
|     2019|        beauty|             1|
|     2019|diy-and-crafts|             2|
|     2019|     education|             2|
|     2019|       finance|             1|
|     2019|  mens-fashion|             1|
|     2019|      vehicles|             1|
|     2020|           art|             1|
|     2020|        beauty|             2|
+---------+--------------+--------

In [0]:
# Select the required columns
df_query_step1 = df_combined.select("country", "poster_name", "follower_count")

# Define a window partitioned by country and ordered by follower_count in descending order
window_spec = Window.orderBy(col("follower_count").desc())

# Rank users globally based on follower_count
df_ranked = df_query_step1.withColumn("rank", rank().over(window_spec))

# Filter the top-ranked user
df_top_follower = df_ranked.filter(col("rank") == 1).select("country", "follower_count")

# Show the result
df_top_follower.show()

+--------------+--------------+
|       country|follower_count|
+--------------+--------------+
|American Samoa|       5000000|
|American Samoa|       5000000|
+--------------+--------------+



In [0]:
df_user_pin = df_pin_cleaned.join(df_user_cleaned,how="inner",on="ind")
# Step 1: Create an age_group column
df_with_age_group = df_user_pin.withColumn(
    "age_group",
    when((col("age") >= 18) & (col("age") <= 24), "18-24")
    .when((col("age") >= 25) & (col("age") <= 35), "25-35")
    .when((col("age") >= 36) & (col("age") <= 50), "36-50")
    .when(col("age") > 50, "+50")
    .otherwise(None)  # Exclude users with age outside these ranges
)

# Step 2: Group by age_group and category, and count the number of posts
category_counts = df_with_age_group.groupBy("age_group", "category").agg(
    count("*").alias("category_count")
)

# Step 3: Rank categories within each age group
window_spec = Window.partitionBy("age_group").orderBy(col("category_count").desc())
ranked_categories = category_counts.withColumn("rank", rank().over(window_spec))

# Step 4: Filter for the most popular category in each age group
most_popular_categories = ranked_categories.filter(col("rank") == 1).drop("rank")

# Show the result
most_popular_categories.show()

+---------+--------------+--------------+
|age_group|      category|category_count|
+---------+--------------+--------------+
|      +50|        beauty|             2|
|    18-24|        beauty|             4|
|    18-24|  mens-fashion|             4|
|    25-35|           art|             4|
|    25-35|     christmas|             4|
|    36-50|    home-decor|             2|
|    36-50|        beauty|             2|
|    36-50|      vehicles|             2|
|    36-50|diy-and-crafts|             2|
|    36-50|           art|             2|
+---------+--------------+--------------+



In [0]:
from pyspark.sql.functions import year, col, count

# Step 1: Extract the year from the date_joined column
df_user_with_year = df_user.withColumn("post_year", year("date_joined"))

# Step 2: Filter rows where post_year is between 2015 and 2020
df_filtered = df_user_with_year.filter((col("post_year") >= 2015) & (col("post_year") <= 2020))

# Step 3: Group by post_year and count the number of users
df_users_joined = df_filtered.groupBy("post_year").agg(
    count("*").alias("number_users_joined")
)

# Step 4: Order the results by post_year
df_users_joined = df_users_joined.orderBy("post_year")

# Show the results
df_users_joined.show()


+---------+-------------------+
|post_year|number_users_joined|
+---------+-------------------+
|     2015|                 20|
|     2016|                 32|
|     2017|                  8|
+---------+-------------------+



In [0]:
# Step 1: Extract the year from the timestamp column
df_combined_with_year = df_combined.withColumn("post_year", year("timestamp"))

# Step 2: Filter rows where post_year is between 2015 and 2020
df_filtered = df_combined_with_year.filter((col("post_year") >= 2015) & (col("post_year") <= 2020))

# Step 3: Calculate the median follower count for each year
results = []
post_years = df_filtered.select("post_year").distinct().collect()  # Get unique years

for row in post_years:
    post_year = row["post_year"]
    
    # Filter data for the specific year
    df_year = df_filtered.filter(col("post_year") == post_year)
    
    # Calculate the median using approxQuantile
    median_follower_count = df_year.approxQuantile("follower_count", [0.5], 0.01)[0]
    
    # Append the result to the list
    results.append((post_year, median_follower_count))

# Step 4: Create a new DataFrame for the results
schema = StructType([
    StructField("post_year", IntegerType(), True),
    StructField("median_follower_count", DoubleType(), True)
])

df_median_follower_count = spark.createDataFrame(results, schema)

# Order the DataFrame by post_year
df_median_follower_count = df_median_follower_count.orderBy("post_year")

# Show the result
df_median_follower_count.show()

+---------+---------------------+
|post_year|median_follower_count|
+---------+---------------------+
|     2017|             613000.0|
|     2018|              22000.0|
|     2019|              22000.0|
|     2020|               9000.0|
+---------+---------------------+



In [0]:
df_combined_with_user = df_combined_with_year.join(df_user_cleaned,how="inner",on="ind")

# Step 1: Create an age_group column
df_with_age_group = df_combined_with_user.withColumn(
    "age_group",
    when((col("age") >= 18) & (col("age") <= 24), "18-24")
    .when((col("age") >= 25) & (col("age") <= 35), "25-35")
    .when((col("age") >= 36) & (col("age") <= 50), "36-50")
    .when(col("age") > 50, "+50")
    .otherwise(None)  # Exclude rows with invalid age
)

# Step 2: Extract post_year from timestamp
df_with_year = df_with_age_group.withColumn("post_year", year("timestamp"))

# Step 3: Filter rows where post_year is between 2015 and 2020
df_filtered = df_with_year.filter((col("post_year") >= 2015) & (col("post_year") <= 2020))

# Step 4: Calculate median follower count for each age_group and post_year
results = []
age_post_years = df_filtered.select("age_group", "post_year").distinct().collect()  # Get unique age_group and post_year combinations

for row in age_post_years:
    age_group = row["age_group"]
    post_year = row["post_year"]
    
    # Filter data for the specific age_group and post_year
    df_group = df_filtered.filter((col("age_group") == age_group) & (col("post_year") == post_year))
    
    # Calculate the median using approxQuantile
    median_follower_count = df_group.approxQuantile("follower_count", [0.5], 0.01)[0]
    
    # Append the result to the list
    results.append((age_group, post_year, median_follower_count))

# Step 5: Create a new DataFrame for the results
schema = StructType([
    StructField("age_group", StringType(), True),
    StructField("post_year", IntegerType(), True),
    StructField("median_follower_count", DoubleType(), True)
])

df_median_follower_count = spark.createDataFrame(results, schema)

# Order the DataFrame by age_group and post_year
df_median_follower_count = df_median_follower_count.orderBy("age_group", "post_year")

# Show the result
df_median_follower_count.show()


+---------+---------+---------------------+
|age_group|post_year|median_follower_count|
+---------+---------+---------------------+
|      +50|     2017|             326000.0|
|      +50|     2018|               9000.0|
|      +50|     2020|                 20.0|
|    18-24|     2017|             613000.0|
|    18-24|     2018|              28000.0|
|    18-24|     2019|              22000.0|
|    25-35|     2018|              22000.0|
|    25-35|     2019|               3000.0|
|    25-35|     2020|              52000.0|
|    36-50|     2018|                  0.0|
|    36-50|     2019|              89000.0|
|    36-50|     2020|                314.0|
+---------+---------+---------------------+

